In [ ]:
import os
from os import path

# Third-party
from astropy.constants import G
from astropy.io import fits, ascii
from astropy.stats import median_absolute_deviation, LombScargle
from astropy.time import Time
from astropy.table import Table, QTable, join
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
import tqdm
from scipy.stats import beta, binned_statistic

from thejoker import JokerSamples

from twoface.config import TWOFACE_CACHE_PATH
from twoface.samples_analysis import MAP_sample
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, NessRG,
                        StarResult, Status, JokerRun)
from twoface.plot import plot_two_panel, plot_phase_fold
from twoface.mass import get_m2_min, mf, period_at_surface, asini, stellar_radius
from twoface.log import log as logger
logger.setLevel(100)
from twoface.ext import hatlc

In [ ]:
plot_path = '../paper/figures/'
# table_path = '../data/'
table_path = '../../twoface/paper/1-catalog/tables/'

In [ ]:
unimodal = QTable.read(path.join(table_path, 'highK-unimodal.fits'), character_as_bytes=False)
clean_uni = unimodal[ (unimodal['clean_flag'] == 0) & (unimodal['converged'])]
high_logg = clean_uni[clean_uni['LOGG'] > 2]
print('{0} unimodal, {1} clean, {2} high logg'
      .format(len(unimodal), len(clean_uni), len(high_logg)))

In [ ]:
hat = ascii.read('../../twoface/data/hatnet-apogee-xmatch/hatnet-apogee-lcmatches.csv')
hat.rename_column('apogee_id', 'APOGEE_ID')
catalog = join(high_logg, hat, keys='APOGEE_ID')

In [ ]:
for row in catalog:
    fn = '../../twoface/data/hatnet-apogee-xmatch/{0}-V0-DR0-hatlc.sqlite.gz'.format(row['hatid'])

    lc, msg = hatlc.read_and_filter_sqlitecurve(fn)

    t = Time(lc['bjd'], format='jd', scale='tcb')

    normlcd = hatlc.normalize_lcdict(lc)
    mags, errs = normlcd['atf_000'], normlcd['aie_000']
    
    clean_mask = np.isfinite(mags) & np.isfinite(errs)
    t = t[clean_mask]
    mags = mags[clean_mask]
    errs = errs[clean_mask]
    
    if len(mags) == 0:
        continue
        
    ls = LombScargle(t.mjd, mags, errs)
    frequency, power = ls.autopower()
    P = 1 / frequency[power.argmax()]
    print(P, row['P'], row['P_err'])
    
    fig, axes = plt.subplots(3, 1, figsize=(8, 12), sharey=True)
    
    ax = axes[0]
    for flt in np.unique(lc['flt'][clean_mask]):
        mask = lc['flt'][clean_mask] == flt
        ax.errorbar(t.mjd[mask], mags[mask], errs[mask], 
                    linestyle='none', marker='.', alpha=0.1, )
    
    ax = axes[1]
    for flt in np.unique(lc['flt'][clean_mask]):
        mask = lc['flt'][clean_mask] == flt
        ax.errorbar((t.mjd[mask] / row['P'].value) % 1., mags[mask], errs[mask], 
                    linestyle='none', marker='.', alpha=0.1, )
        
    ax = axes[2]
    ax.errorbar((t.mjd / P) % 1., mags, errs, 
                linestyle='none', marker='.', alpha=0.1, )
    
    med = np.nanmedian(mags)
    mad = median_absolute_deviation(mags, ignore_nan=True)
    ax.set_ylim(med + 0.2, med - 0.2)